# Improved Real Estate Price Prediction Preprocessing
## Tunisian Real Estate Dataset

**Key Improvements:**
1. Separate pipelines for rent and sale transactions
2. Better handling of high-cardinality features (region)
3. Feature engineering (price per sqm, region aggregations)
4. Proper treatment of ordinal features
5. Rare category grouping

In [40]:
%pip install scikit-learn
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Load and Explore Data

In [41]:
# Resolve data path from workspace
def find_data_path():
    candidates = [
        Path.cwd() / "data" / "processed" / "cleaned_real_estate.csv",
        Path.cwd().parent / "data" / "processed" / "cleaned_real_estate.csv",
        Path.cwd().parent / "ML" / "data" / "processed" / "cleaned_real_estate.csv",
        Path.home() / "Desktop" / "ML-project" / "ML" / "data" / "processed" / "cleaned_real_estate.csv",
    ]
    for p in candidates:
        if p.exists():
            print(f"Found data at: {p}")
            return p
    print("Checked paths:")
    for p in candidates:
        print(f"  - {p}")
    raise FileNotFoundError("cleaned_real_estate.csv not found in expected locations")

DATA_PATH = find_data_path()
df = pd.read_csv(DATA_PATH)

print(f"Total records: {len(df):,}")
print(f"\nTransaction distribution:")
print(df['transaction'].value_counts())
print(f"\nRent: {len(df[df['transaction']=='rent']):,} ({len(df[df['transaction']=='rent'])/len(df)*100:.1f}%)")
print(f"Sale: {len(df[df['transaction']=='sale']):,} ({len(df[df['transaction']=='sale'])/len(df)*100:.1f}%)")

Found data at: C:\Users\Mediatek\Desktop\ML-project\ML\data\processed\cleaned_real_estate.csv
Total records: 9,744

Transaction distribution:
transaction
rent    5636
sale    4108
Name: count, dtype: int64

Rent: 5,636 (57.8%)
Sale: 4,108 (42.2%)


## 2. Feature Engineering

Creating new features that will help the model:

In [42]:
# Basic cleaning
required_cols = ['price', 'surface', 'rooms', 'bathrooms', 'region', 'city', 'property_type', 'transaction']
df = df.dropna(subset=required_cols).copy()
df = df[(df['price'] > 0) & (df['surface'] > 0)]

def cap_outliers_iqr(series, k=1.5):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - k * iqr
    upper = q3 + k * iqr
    return series.clip(lower, upper)

# Cap extreme outliers to stabilize training
for col in ['price', 'surface', 'rooms', 'bathrooms']:
    df[col] = cap_outliers_iqr(df[col])

# Feature 1: Total rooms (rooms + bathrooms)
df['total_rooms'] = df['rooms'] + df['bathrooms']

# Feature 2: Surface category (small, medium, large)
df['surface_category'] = pd.cut(df['surface'],
                                  bins=[0, 80, 150, 300, 10000],
                                  labels=['small', 'medium', 'large', 'very_large'])

# Feature 3: Group rare regions (< 20 occurrences) into 'other'
# The REGION itself matters (e.g., "Lac" is expensive, "Hay Tahrir" is cheaper)
# We'll use target encoding to convert region names to their mean prices
region_counts = df['region'].value_counts()
df['region_grouped'] = df['region'].copy()
rare_regions = region_counts[region_counts < 20].index
df.loc[df['region'].isin(rare_regions), 'region_grouped'] = 'other_region'

# Feature 4: Log transforms for skewed variables
df['log_surface'] = np.log1p(df['surface'])
df['log_total_rooms'] = np.log1p(df['total_rooms'])

print(f"Original regions: {df['region'].nunique()}")
print(f"Grouped regions: {df['region_grouped'].nunique()}")
print(f"Rare regions grouped: {len(rare_regions)}")

Original regions: 267
Grouped regions: 100
Rare regions grouped: 168


## 3. Target Encoding for High Cardinality Features

For regions, we'll create mean-encoded features (average price by region):

In [43]:
def create_target_encoding(df, column, target, min_samples=10, smoothing=10):
    """
    Create target encoding with smoothing for rare categories.
    Returns the mapping dictionary and global mean.
    """
    global_mean = df[target].mean()
    agg = df.groupby(column)[target].agg(['mean', 'count'])
    agg['count'] = agg['count'].clip(lower=min_samples)
    agg['smoothed_mean'] = (
        (agg['count'] * agg['mean'] + smoothing * global_mean) /
        (agg['count'] + smoothing)
    )
    return agg['smoothed_mean'].to_dict(), global_mean

def add_kfold_target_encoding(X, y, column, n_splits=5, min_samples=10, smoothing=10):
    """
    Out-of-fold target encoding for training data, plus full mapping for test.
    """
    oof_encoded = pd.Series(index=X.index, dtype=float)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    for train_idx, val_idx in kf.split(X):
        train_fold = pd.concat([X.iloc[train_idx], y.iloc[train_idx]], axis=1)
        mapping, global_mean = create_target_encoding(
            train_fold, column, y.name, min_samples=min_samples, smoothing=smoothing
        )
        oof_encoded.iloc[val_idx] = X.iloc[val_idx][column].map(mapping).fillna(global_mean)
    full_mapping, full_global = create_target_encoding(
        pd.concat([X, y], axis=1), column, y.name, min_samples=min_samples, smoothing=smoothing
    )
    return oof_encoded, full_mapping, full_global

## 4. Split Data by Transaction Type

**Critical Step:** Separate rent and sale data for independent modeling:

In [44]:
# Split by transaction type
df_rent = df[df['transaction'] == 'rent'].copy()
df_sale = df[df['transaction'] == 'sale'].copy()

print(f"Rent dataset: {len(df_rent):,} records")
print(f"Sale dataset: {len(df_sale):,} records")

# Drop transaction column as it's no longer needed
df_rent = df_rent.drop(columns=['transaction'])
df_sale = df_sale.drop(columns=['transaction'])

Rent dataset: 5,636 records
Sale dataset: 4,108 records


## 5. Create Target Encodings (Separate for Rent and Sale)

In [45]:
# We'll create these encodings on the training set only (to avoid data leakage)
TARGET = 'price'

# Define feature groups
CATEGORICAL_COLS = [
    'property_type',
    'city',
    'surface_category'
]

NUMERICAL_COLS = [
    'surface',
    'log_surface',
    'log_total_rooms'
]

# Treat rooms and bathrooms as ordinal (order matters)
ORDINAL_COLS = [
    'rooms',
    'bathrooms',
    'total_rooms'
]

# High cardinality features for target encoding
# REGION will be target-encoded to its mean price (most important feature!)
TARGET_ENCODE_COLS = [
    'region_grouped'
]

ALL_FEATURES = CATEGORICAL_COLS + NUMERICAL_COLS + ORDINAL_COLS + TARGET_ENCODE_COLS

print(f"Total features: {len(ALL_FEATURES)}")
print(f"Categorical: {CATEGORICAL_COLS}")
print(f"Numerical: {NUMERICAL_COLS}")
print(f"Ordinal: {ORDINAL_COLS}")
print(f"Target encoded (REGION): {TARGET_ENCODE_COLS}")

Total features: 10
Categorical: ['property_type', 'city', 'surface_category']
Numerical: ['surface', 'log_surface', 'log_total_rooms']
Ordinal: ['rooms', 'bathrooms', 'total_rooms']
Target encoded (REGION): ['region_grouped']


## 6. Train-Test Split (Separate for Each Transaction Type)

In [46]:
def stratified_split(X, y, test_size=0.2, random_state=42, n_bins=10):
    try:
        y_binned = pd.qcut(y, q=n_bins, duplicates='drop')
        return train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=y_binned
        )
    except ValueError:
        return train_test_split(
            X, y, test_size=test_size, random_state=random_state
        )

# RENT data split
X_rent = df_rent[ALL_FEATURES]
y_rent = df_rent[TARGET]

X_rent_train, X_rent_test, y_rent_train, y_rent_test = stratified_split(
    X_rent, y_rent, test_size=0.2, random_state=42, n_bins=10
 )

print("RENT SPLIT:")
print(f"  Train: {len(X_rent_train):,} samples")
print(f"  Test:  {len(X_rent_test):,} samples")

# SALE data split
X_sale = df_sale[ALL_FEATURES]
y_sale = df_sale[TARGET]

X_sale_train, X_sale_test, y_sale_train, y_sale_test = stratified_split(
    X_sale, y_sale, test_size=0.2, random_state=42, n_bins=10
 )

print("\nSALE SPLIT:")
print(f"  Train: {len(X_sale_train):,} samples")
print(f"  Test:  {len(X_sale_test):,} samples")

RENT SPLIT:
  Train: 4,508 samples
  Test:  1,128 samples

SALE SPLIT:
  Train: 3,286 samples
  Test:  822 samples


## 7. Fit Target Encodings on Training Data Only

In [47]:
# Create target encodings for RENT (out-of-fold)
rent_region_oof, rent_region_mapping, rent_region_global = add_kfold_target_encoding(
    X_rent_train, y_rent_train, 'region_grouped', n_splits=5, min_samples=10, smoothing=10
 )

# Create target encodings for SALE (out-of-fold)
sale_region_oof, sale_region_mapping, sale_region_global = add_kfold_target_encoding(
    X_sale_train, y_sale_train, 'region_grouped', n_splits=5, min_samples=10, smoothing=10
 )

print("Target encodings created successfully!")
print(f"\nRent - example region encodings:")
for region, value in list(rent_region_mapping.items())[:5]:
    print(f"  {region}: {value:.2f} TND")
    
print(f"\nSale - example region encodings:")
for region, value in list(sale_region_mapping.items())[:5]:
    print(f"  {region}: {value:.2f} TND")

Target encodings created successfully!

Rent - example region encodings:
  Ain Zaghouan Nord: 1542.03 TND
  Ain Zaghouan Sud: 1300.10 TND
  Ain Zaghouen: 1274.61 TND
  Akouda: 1283.64 TND
  Ariana: 1121.10 TND

Sale - example region encodings:
  Ain Zaghouan Nord: 394765.71 TND
  Ain Zaghouan Sud: 487365.08 TND
  Ain Zaghouen: 317732.81 TND
  Akouda: 469138.89 TND
  Ariana: 413266.67 TND


## 8. Apply Target Encodings

In [ ]:
# Apply to RENT data
X_rent_train = X_rent_train.copy()
X_rent_test = X_rent_test.copy()
X_rent_train['region_encoded'] = rent_region_oof
X_rent_test['region_encoded'] = X_rent_test['region_grouped'].map(rent_region_mapping).fillna(rent_region_global)

# Apply to SALE data
X_sale_train = X_sale_train.copy()
X_sale_test = X_sale_test.copy()
X_sale_train['region_encoded'] = sale_region_oof
X_sale_test['region_encoded'] = X_sale_test['region_grouped'].map(sale_region_mapping).fillna(sale_region_global)

# Drop the original region column
X_rent_train = X_rent_train.drop(columns=['region_grouped'])
X_rent_test = X_rent_test.drop(columns=['region_grouped'])
X_sale_train = X_sale_train.drop(columns=['region_grouped'])
X_sale_test = X_sale_test.drop(columns=['region_grouped'])

# Update feature lists - add region_encoded to numerical
NUMERICAL_COLS.append('region_encoded')
print("Target encoding applied!")
print(f"\nUpdated numerical columns: {NUMERICAL_COLS}")

Target encoding + advanced feature engineering applied!

New features added: surface_x_rooms, surface_x_region, rooms_x_bathrooms, surface_squared
Total numerical columns: 8


## 9. Create Preprocessing Pipelines

In [49]:
# Create the same preprocessor for both rent and sale
def create_preprocessor():
    return ColumnTransformer(
        transformers=[
            (
                'categorical',
                OneHotEncoder(handle_unknown='ignore', sparse_output=False),
                CATEGORICAL_COLS
            ),
            (
                'numerical',
                StandardScaler(),
                NUMERICAL_COLS
            ),
            (
                'ordinal',
                StandardScaler(),  # Or keep as-is, depending on your preference
                ORDINAL_COLS
            )
        ]
    )

preprocessor_rent = create_preprocessor()
preprocessor_sale = create_preprocessor()

print("Preprocessing pipelines created!")

Preprocessing pipelines created!


## 10. Fit and Transform Data

In [50]:
# RENT preprocessing
X_rent_train_prepared = preprocessor_rent.fit_transform(X_rent_train)
X_rent_test_prepared = preprocessor_rent.transform(X_rent_test)

print("RENT data preprocessed:")
print(f"  Train shape: {X_rent_train_prepared.shape}")
print(f"  Test shape:  {X_rent_test_prepared.shape}")

# SALE preprocessing
X_sale_train_prepared = preprocessor_sale.fit_transform(X_sale_train)
X_sale_test_prepared = preprocessor_sale.transform(X_sale_test)

print("\nSALE data preprocessed:")
print(f"  Train shape: {X_sale_train_prepared.shape}")
print(f"  Test shape:  {X_sale_test_prepared.shape}")

# Verify shapes match for train/test pairs
assert X_rent_train_prepared.shape[1] == X_rent_test_prepared.shape[1], "RENT: Train/Test feature mismatch!"
assert X_sale_train_prepared.shape[1] == X_sale_test_prepared.shape[1], "SALE: Train/Test feature mismatch!"
print("\n✅ Feature shapes verified - no mismatches!")

RENT data preprocessed:
  Train shape: (4508, 42)
  Test shape:  (1128, 42)

SALE data preprocessed:
  Train shape: (3286, 42)
  Test shape:  (822, 42)

✅ Feature shapes verified - no mismatches!


## 11. Save Preprocessed Data and Pipelines

In [ ]:
# Create output directories - explicitly under ML/data/
PREP_VERSION = "v2"

# Direct path to ML directory
ML_DIR = Path("C:/Users/Mediatek/Desktop/ML-project/ML")

# Verify the directory exists
if not ML_DIR.exists():
    print(f"⚠️  Path does not exist: {ML_DIR}")
    print("Trying alternate paths...")
    candidates = [
        Path.cwd(),
        Path.cwd().parent,
        Path.cwd().parent / "ML",
        Path.home() / "Desktop" / "ML-project" / "ML"
    ]
    for p in candidates:
        if (p / "data" / "processed" / "cleaned_real_estate.csv").exists():
            ML_DIR = p
            break

print(f"✅ ML directory: {ML_DIR}")
OUTPUT_DIR = ML_DIR

# Create directories
os.makedirs(OUTPUT_DIR / f"data/prepared_{PREP_VERSION}/rent", exist_ok=True)
os.makedirs(OUTPUT_DIR / f"data/prepared_{PREP_VERSION}/sale", exist_ok=True)
os.makedirs(OUTPUT_DIR / f"data/preprocessing_{PREP_VERSION}", exist_ok=True)

print(f"\n📁 Output paths:")
print(f"  prepared_{PREP_VERSION}: {OUTPUT_DIR / f'data/prepared_{PREP_VERSION}'}")
print(f"  preprocessing_{PREP_VERSION}: {OUTPUT_DIR / f'data/preprocessing_{PREP_VERSION}'}")

# Save RENT data
np.save(OUTPUT_DIR / f"data/prepared_{PREP_VERSION}/rent/X_train.npy", X_rent_train_prepared)
np.save(OUTPUT_DIR / f"data/prepared_{PREP_VERSION}/rent/X_test.npy", X_rent_test_prepared)
np.save(OUTPUT_DIR / f"data/prepared_{PREP_VERSION}/rent/y_train.npy", y_rent_train.to_numpy())
np.save(OUTPUT_DIR / f"data/prepared_{PREP_VERSION}/rent/y_test.npy", y_rent_test.to_numpy())

# Save SALE data
np.save(OUTPUT_DIR / f"data/prepared_{PREP_VERSION}/sale/X_train.npy", X_sale_train_prepared)
np.save(OUTPUT_DIR / f"data/prepared_{PREP_VERSION}/sale/X_test.npy", X_sale_test_prepared)
np.save(OUTPUT_DIR / f"data/prepared_{PREP_VERSION}/sale/y_train.npy", y_sale_train.to_numpy())
np.save(OUTPUT_DIR / f"data/prepared_{PREP_VERSION}/sale/y_test.npy", y_sale_test.to_numpy())

# Save preprocessors
joblib.dump(
    preprocessor_rent,
    OUTPUT_DIR / f"data/preprocessing_{PREP_VERSION}/preprocessor_rent.joblib"
 )
joblib.dump(
    preprocessor_sale,
    OUTPUT_DIR / f"data/preprocessing_{PREP_VERSION}/preprocessor_sale.joblib"
 )

# Save target encodings
joblib.dump(
    {'region_encoding': rent_region_mapping, 'region_global': rent_region_global},
    OUTPUT_DIR / f"data/preprocessing_{PREP_VERSION}/target_encodings_rent.joblib"
 )
joblib.dump(
    {'region_encoding': sale_region_mapping, 'region_global': sale_region_global},
    OUTPUT_DIR / f"data/preprocessing_{PREP_VERSION}/target_encodings_sale.joblib"
 )

print("\n✅ All data and pipelines saved successfully!")
print(f"\nFiles saved:")
print(f"  Rent prepared: {OUTPUT_DIR / f'data/prepared_{PREP_VERSION}/rent/'}")
print(f"  Sale prepared: {OUTPUT_DIR / f'data/prepared_{PREP_VERSION}/sale/'}")
print(f"  Preprocessing: {OUTPUT_DIR / f'data/preprocessing_{PREP_VERSION}/'}")

✅ All data and pipelines saved successfully!

Output directory: C:\Users\Mediatek

Files saved:
  - Rent: X_train, X_test, y_train, y_test
  - Sale: X_train, X_test, y_train, y_test
  - Preprocessors: preprocessor_rent.joblib, preprocessor_sale.joblib
  - Target encodings: target_encodings_rent.joblib, target_encodings_sale.joblib


## 12. Summary Statistics

In [ ]:
print("=" * 70)
print("PREPROCESSING SUMMARY")
print("=" * 70)

print("\n📊 DATASET SIZES:")
print(f"  Rent Train:  {X_rent_train_prepared.shape[0]:,} samples x {X_rent_train_prepared.shape[1]} features")
print(f"  Rent Test:   {X_rent_test_prepared.shape[0]:,} samples x {X_rent_test_prepared.shape[1]} features")
print(f"  Sale Train:  {X_sale_train_prepared.shape[0]:,} samples x {X_sale_train_prepared.shape[1]} features")
print(f"  Sale Test:   {X_sale_test_prepared.shape[0]:,} samples x {X_sale_test_prepared.shape[1]} features")

print("\n💰 PRICE STATISTICS:")
print("  RENT:")
print(f"    Train mean: {y_rent_train.mean():.2f} TND (std: {y_rent_train.std():.2f})")
print(f"    Test mean:  {y_rent_test.mean():.2f} TND (std: {y_rent_test.std():.2f})")
print(f"    Price range: {y_rent_train.min():.0f} - {y_rent_train.max():.0f} TND")
print("  SALE:")
print(f"    Train mean: {y_sale_train.mean():.2f} TND (std: {y_sale_train.std():.2f})")
print(f"    Test mean:  {y_sale_test.mean():.2f} TND (std: {y_sale_test.std():.2f})")
print(f"    Price range: {y_sale_train.min():.0f} - {y_sale_train.max():.0f} TND")

print("\n🔧 FEATURE ENGINEERING:")
print(f"  Original features: 7")
print(f"  Engineered features: {len(ALL_FEATURES) - 7}")
print(f"  Total before encoding: {len(ALL_FEATURES)}")
print(f"  Total after one-hot: {X_rent_train_prepared.shape[1]}")

print("\n✅ Ready for model training!")

PREPROCESSING SUMMARY

📊 DATASET SIZES:
  Rent Train:  4,508 samples x 42 features
  Rent Test:   1,128 samples x 42 features
  Sale Train:  3,286 samples x 42 features
  Sale Test:   822 samples x 42 features

💰 PRICE STATISTICS:
  RENT:
    Train mean: 1458.95 TND (std: 1367.79)
    Test mean:  1499.33 TND (std: 1486.87)
    Price range: 200 - 15000 TND
  SALE:
    Train mean: 406366.68 TND (std: 210805.66)
    Test mean:  406126.73 TND (std: 211631.32)
    Price range: 20000 - 748500 TND

🔧 FEATURE ENGINEERING:
  Base features: 7
  After feature engineering: 10
  After one-hot encoding: 42 features (rent)
  Advanced features: surface_x_rooms, surface_x_region, rooms_x_bathrooms, surface_squared

⚠️  NOTE:
  R² scores of 0.55-0.62 indicate:
  - Missing important features (condition, amenities, location proximity)
  - High inherent price variance in the data
  - Consider collecting more detailed property attributes

✅ Ready for model training with enhanced features!


## Next Steps

Now you're ready to train separate models:

1. **For RENT model:**
   - Load: `X_rent_train.npy`, `y_rent_train.npy`
   - Train models (Linear Regression, Random Forest, XGBoost, etc.)
   - Evaluate on `X_rent_test.npy`, `y_rent_test.npy`

2. **For SALE model:**
   - Load: `X_sale_train.npy`, `y_sale_train.npy`
   - Train models (Linear Regression, Random Forest, XGBoost, etc.)
   - Evaluate on `X_sale_test.npy`, `y_sale_test.npy`

3. **For predictions on new data:**
   - Load the appropriate preprocessor (`preprocessor_rent.joblib` or `preprocessor_sale.joblib`)
   - Load target encodings
   - Apply the same feature engineering
   - Transform with the preprocessor
   - Predict with your trained model